<a href="https://colab.research.google.com/github/princesapkota/3rdsemAiworksheet4/blob/main/worksheet4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd

In [5]:
#problem 1 start
from google.colab import drive
drive.mount('/content/drive/')
df = pd.read_csv("/content/drive/MyDrive/datasets/diabetes.csv")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
df.head()
#getting the first 5 rows of dataframe

In [ ]:
df.info()
#shwoing the information of dataframe

In [ ]:
df.describe()
#showing detailed columns and row data of dataframe

In [ ]:
df.isnull().sum()
#showing a whole list of null valued datas

In [ ]:
#handling missing values
# Checking for zero or anomalous values in specific columns
columns_to_check = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in columns_to_check:
    print(f"\n{col} - Zero value count: {sum(df[col] == 0)}")
#Replacing zero values with the column median
for col in columns_to_check:
    df[col] = df[col].replace(0, df[col].median())

df.describe()


In [ ]:
#featuring engineering

x =df.drop(columns=['Outcome']).values
y = df['Outcome'].values
#spl;ittting into traim 70% ro 30%
np.random.seed(42)
indices = np.arange(len(x))
np.random.shuffle(indices)
#defining the splits
splitIndex = int(len(x)*0.7)
xTrain, xTest = x[indices[:splitIndex]], x[indices[splitIndex:]]
yTrain, yTest = y[indices[:splitIndex]], y[indices[splitIndex:]]
print(len(x))
print(len(y))
print(xTrain[:5])
print(yTrain[:5])

In [ ]:
# Function to calculate the Euclidean distance between two points
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2) ** 2))


In [ ]:
# Function to predict the class for a single query point
def predict_single(xTrain, yTrain, query, k=5):
    distances = []

    # Calculate distance from the query to all training points
    for i in range(len(xTrain)):
        distance = euclidean_distance(xTrain[i], query)
        distances.append((distance, yTrain[i]))  # Append (distance, label)

    # Sort the distances
    distances.sort(key=lambda x: x[0])  # Sort by distance

    # Get the labels of the k nearest neighbors
    k_nearest_labels = [label for _, label in distances[:k]]

    # Majority vote: find the most common class
    prediction = max(set(k_nearest_labels), key=k_nearest_labels.count)
    return prediction


In [ ]:
# Function to predict for all test samples
def predict(xTrain, yTrain, X_test, k=5):
    predictions = []
    for query in X_test:
        pred = predict_single(xTrain, yTrain, query, k)
        predictions.append(pred)
    return predictions


In [ ]:

# Function to calculate accuracy
def accuracy_score(yTrue, yPred):
    correct = np.sum(yTrue == yPred)  # Count matching values
    return correct / len(yTrue)
k = 5
yPred = predict(xTrain, yTrain, xTest, k)
accuracy = accuracy_score(yTest, yPred)
print(f"Accuracy of KNN model with k={k}: {accuracy * 100:.2f}%")
print("\nFirst 10 predictions vs actual labels:")
print("Predicted:",yPred[:10])
print("Actual:", yTest[:10])


In [ ]:
#for minimun to maxu=imum scaling
def min_max_scale(X):
    X_min = np.min(X, axis=0)
    X_max = np.max(X, axis=0)
    return (X - X_min) / (X_max - X_min)

xTrainScaled = min_max_scale(xTrain)
xTestScaled = min_max_scale(xTest)
print(xTrainScaled[:5])


In [ ]:
##to training aand testing knn scaled data
kNeighbors = 5
yPredScaled = predict(xTrainScaled, yTrain, xTestScaled, kNeighbors)
accuracyScaled = accuracy_score(yTest, yPredScaled)

print(f"Accuracy with scaled data (k={kNeighbors}): {accuracyScaled * 100:.2f}%")
#comparing
print(f"Accuracy with original data: {accuracy * 100:.2f}%")
print(f"Accuracy with scaled data: {accuracyScaled * 100:.2f}%")


In [ ]:
#PRIBELM 3
#predicting a single query wuith costt
def predict_single_with_cost(xTrain, yTrain, query, k):
    distances = []
    cost = 0  # Track computation cost (distance calculations)

    for i in range(len(xTrain)):
        distance = euclidean_distance(xTrain[i], query)
        distances.append((distance, yTrain[i]))
        cost += 1

    distances.sort(key=lambda x: x[0])
    k_nearest_labels = [label for _, label in distances[:k]]
    prediction = max(set(k_nearest_labels), key=k_nearest_labels.count)

    return prediction, cost


In [ ]:
#prediciting a wholee query with cost
def predict_with_cost(xTrain, yTrain, xTest, k):
    predictions = []
    total_cost = 0

    for query in xTest:
        pred, cost = predict_single_with_cost(xTrain, yTrain, query, k)
        predictions.append(pred)
        total_cost += cost

    return np.array(predictions), total_cost


In [ ]:
#experimenting with k = 1 to 15 values
k_values = range(1, 16)
accuracy_original = []
accuracy_scaled = []
cost_original = []
cost_scaled = []

for k in k_values:
    #original dataset
    yPredOriginal, costOrig = predict_with_cost(xTrain, yTrain, xTest, k)
    accuracy_original.append(accuracy_score(yTest, yPredOriginal))
    cost_original.append(costOrig)

    #scaled dataset
    yPredScaled, costScale = predict_with_cost(xTrainScaled, yTrain, xTestScaled, k)
    accuracy_scaled.append(accuracy_score(yTest, yPredScaled))
    cost_scaled.append(costScale)

print("Accuracies with original data:", accuracy_original)
print("Accuracies with scaled data:", accuracy_scaled)
print("Computation cost with original data:", cost_original)
print("Computation cost with scaled data:", cost_scaled)


In [ ]:
#plotting the values results in graph
#k vs accuray
import matplotlib.pyplot as plt

plt.figure()
plt.plot(k_values, accuracy_original, label='Original Dataset')
plt.plot(k_values, accuracy_scaled, label='Scaled Dataset')
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Accuracy')
plt.title('k vs. Accuracy')
plt.legend()
plt.show()


In [ ]:
#plot k vs computational cost gprah

plt.figure()
plt.plot(k_values, cost_original, label='Original Data Cost')
plt.plot(k_values, cost_scaled, label='Scaled Data Cost')
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Distance Calculations')
plt.title('Computation Cost for Different Numbers of Neighbors')
plt.legend()
plt.grid()
plt.show()


problem 2 discussion
### If the accuracy improves significantly after scaling, it means the original dataset had features with very different ranges, and those differences were messing up the distance calculations. Features with larger values were unfairly dominating the results, making KNN less accurate.

KNN works by measuring distances between points to decide which neighbors are closest. If some features have bigger ranges (like income vs. age), those features will overpower the others in the distance calculations. This can lead to poor neighbor selection and lower accuracy.

### Scaling fixes this by making all features equally important, ensuring KNN focuses on the actual relationships in the data rather than being biased by size differences.